In [ ]:
import pysam
from tqdm import tqdm
import numpy as np
import pandas as pd
'''
dir="/Users/mhryansohn/Desktop/01.Workspace/01.Projects/03.AK1-PacBio/01.DNA/Public_Data/HG002"
hg002 = pysam.Ali
ak1 = pysam.AlignmentFile("chr20_500000-1000000_AK1.bam", 'rb')
a = next(ak1)
b = next(ak1)
a.tags # get all tags into list 
tag_name =  [tag[0] for tag in a.tags]

a.get_tag('MM')
a.get_tag('ML')
a.query_name
'''
'''
The continuous probability range of 0.0 to 1.0 is remapped to the discrete integers 0 to 255 
inclusively in the ML tag. The probability range corresponding to an integer N is N/256 to (N + 1)/256.
'''
'''
#Negative 일 경우
for pos, prob in list(a.modified_bases.values())[0]:
    if a.seq[pos-1: pos+1] != 'CG':
        print('fuck')
#Positive 일 경우
for pos, prob in list(b.modified_bases.values())[0]:
    if b.seq[pos: pos+2] != 'CG':
        print('fuck')

for read in ak1:
    if read.has_tag('MM') and read.has_tag('ML'):
        print('fuck')
'''

def get_read_level_cg(read_segment, pos):
    if read_segment.modified_bases != {}:
        read_alignment_start = read_segment.get_reference_positions()[0]
        read_alignment_end = read_segment.get_reference_positions()[-1]
        if read_alignment_start <= pos[0] and read_alignment_end >= pos[-1]:
            cg_ml = dict(list(read_segment.modified_bases.values())[0]) # read-level cytosine location and ML
            cg_met = list()
            if read_segment.is_reverse:
                cg_ml = {key-1: value for key, value in cg_ml.items()}
            for k,v in read_segment.get_aligned_pairs(): # read-based coordinate and corresponding reference coordinate
                if v in pos:
                    try:
                        cg_met.append(cg_ml[k]/256)
                    except KeyError:
                        cg_met.append(np.nan)
            return np.array(cg_met)

def make_cg_matrix(alignmentfile, refcgtab, coordinate):
    '''alignmentfile = pysam.AlignmentFile("chr20_500000-1000000_AK1.bam", 'rb')'''
    cg_pos = list(map(lambda x: int(x), refcgtab.loc[coordinate]['CG_Pos'].rstrip(';').split(';')))
    chrom = coordinate.split(':')[0]
    start, end = int(coordinate.split(':')[1].split('-')[0]), int(coordinate.split(':')[1].split('-')[1])
    cg_matrix_list = list()
    for read in alignmentfile.fetch(chrom, start, end):
        read_level_cg = get_read_level_cg(read, cg_pos)
        if np.all(read_level_cg != None):
            cg_matrix_list.append(read_level_cg)
#            print(read.query_name, read.is_reverse)
    return np.array(cg_matrix_list)

def remove_nan_column_from_cg_matrix(matrix):
    mask = np.isnan(matrix)
    mask = ~mask.any(axis=0)
    matrix = matrix[:, mask]
    return matrix

ak1 = pysam.AlignmentFile("chr20_500000-1000000_AK1.bam", 'rb')
ref_10kb_cg = pd.read_table("10kb_CG_Information_hg38_primary_chrX.tab", index_col=0)

test_matrix1 = remove_nan_column_from_cg_matrix(make_cg_matrix(ak1, ref_10kb_cg, 'chr20:500000-510000'))
test_matrix2 = remove_nan_column_from_cg_matrix(make_cg_matrix(ak1, ref_10kb_cg, 'chr20:510000-520000'))

for i in range(test_matrix1.shape[0]):
    for j in range(i+1, matrix.shape[0]):
        vec_i = test_matrix1[i, :]
        vec_j = test_matrix1[j, :]
        euclidean_dist = distance.euclidean(vec_i, vec_j)
        cosine_sim = 1 - distance.cosine(vec_i, vec_j)
        pearson_corr, _ = pearsonr(vec_i, vec_j)
        spearman_corr, _ = spearmanr(vec_i, vec_j)
        print(f"Distance Metrics between vector {i+1} and vector {j+1}: {euclidean_dist}, {cosine_sim}, {pearson_corr}, {spearman_corr}")

# Test
full_length_count = list() # (#read, #CG), #CG for chekc with ref_10kb_cg

for i in ref_10kb_cg[ref_10kb_cg.index.str.startswith('chr20')].index:
    cg_mat = make_cg_matrix(ak1, ref_10kb_cg, i)
    full_length_count.append(np.shape(cg_mat))

full_length_count = list() # (#read, #CG), #CG for chekc with ref_10kb_cg
for i in ref_10kb_cg.iloc[260800:260850].index:
    cg_mat = make_cg_matrix(ak1, ref_10kb_cg, i)
    full_length_count.append(np.shape(cg_mat))
    
for i,v in enumerate(full_length_count):
    if v[1] != ref_10kb_cg.iloc[260800:260850].iloc[i]['CG_Num']:
        print('fuck')


'''for i in cg_pos:'''
    
